# Pico SCPI labTool (PST)

Want to work with a real, non-simulated instrument, but don't have high-end lab equipment on hand? Try a *****PST*****.

Pico SCPI labTool (PST) is an ordinary,
$4 Raspberry Pi Pico
with specialized firmware from https://github.com/michaelstoops/pico_scpi_usbtmc_labtool.
It works on Pico 1 and 1W,
and may work on other RP2040 boards.

It does not currently work on the Pico 2 series based on the RP2350 chip.


## Setup
To get a PST:
1. Acquire a [Raspberry Pi Pico](https://www.raspberrypi.com/products/raspberry-pi-pico/) or compatible device.
2. Acquire [the firmware (~80 KB on GitHub)](https://github.com/michaelstoops/pico_scpi_usbtmc_labtool/releases/download/v0.01/pico_scpi_usbtmc_labtool.uf2)
This code was tested against v0.01.
3. Hold down the device's `BOOTSEL` button while plugging the Pico into USB.
It will appear in your system as a USB mass storage device called RPI-RP2.
4. Copy the firmware image to RPI-RP2.
The device will reboot.
Ignore your system's complaint about unsafely removing the device.
5. Your device is now a PST instrument.
You can power it up and down whenever you want.
The firmware is stored in FLASH ROM and will remain until you change it with the BOOTSEL procedure.

You should be able to find the PST on your system under the name "Pico SCPI labTool" (product ID 0x41C0),
manufacturer "Pico" (vendor ID 0x1209).
It implements the USB Test and Measurement Class,
so it should work with any VISA library.

The testing device appeared in NI VISA Interactive Control on MacOS as `USB0::0x1209::0x41C0::E660583883555B31::INSTR`.
Your device should have a similar ID,
but the fourth field is based on an internal hardware ID and will vary.


## Breadboard (Optional)

PST only requires the Pico board for basic functionality.
However, a breadboard is recommended for setting inputs and visualizing outputs.
Here's a [materials list](fritzing/Pico_PST.fz_bom.html).

Here is a breadboard configuration that connects all four digital inputs to pushbutton switches.
When a button is pressed, it raises the respective input to high.
The four digital outputs are connected to current-limiting resistors and LEDs.

The arrangement of the LEDs is harder to see in the breadboard rendering,
but they are meant to be arranged in the shape of a Microsoft logo.
Note that the long lead of an LED is the cathode and connected to the resistor.
The short leads of the LEDs overlap and are grounded.

![image of a breadboard with Raspberry Pi Pico, four button switches, and four LEDs of red, green, blue, and yellow arranged like a Microsoft logo](fritzing/Pico_PST.fz_bb.svg)

Here is the same circuit in schematic form, which is clearer about the LEDs connections:

![schematic of a breadboard with Raspberry Pi Pico, four button switches, and four LEDs](fritzing/Pico_PST.fz_schem.svg)


## No Breadboard?

If you don't have a breadboard available,
you can probe an output using a voltmeter with COM (ground probe) on the shell of the USB connector.

There is a very helpful pinout diagram at https://pico.pinout.xyz/.

You can also raise an input by touching either lead of a 3k ohm (or higher) resistor to the input pin and the 3.3V rail on pin 36.
It's possible to use a plain wire if you don't have a resistor,
but a resistor is safer.
In the case that the lead touches a low output or supply pin,
the resistor will limit current,
where a wire would burn out the GPIO pin or short the power supply.


## Create the Instrument

With your PST plugged in and SCPI ID in hand, let's try it out!

In [1]:
from time import sleep

from qcodes_contrib_drivers.drivers.Pico.Pico_PST import PicoPST

# Your device serial number will be different v--------------v
pico = PicoPST('pico', 'USB0::0x1209::0x41C0::E660583883555B31::INSTR')

Connected to: PICO PST (serial:E660583883555B31, firmware:01.00) in 0.04s


## Parameters

Let's see what the parameters say they can do.

In [5]:
for io in [
    pico.output_gp19,
    pico.output_gp20,
    pico.output_gp21,
    pico.output_gp22,
    pico.input_gp10,
    pico.input_gp11,
    pico.input_gp12,
    pico.input_gp13,
]:
    print(f"{io}: {io.__doc__}")

pico_output_gp19: Digital OUTput GP19 (Pico pin 25)
pico_output_gp20: Digital OUTput GP20 (Pico pin 26)
pico_output_gp21: Digital OUTput GP21 (Pico pin 27)
pico_output_gp22: Digital OUTput GP22 (Pico pin 29)
pico_input_gp10: Digital INput GP10 (Pico pin 14)
pico_input_gp11: Digital INput GP11 (Pico pin 15)
pico_input_gp12: Digital INput GP12 (Pico pin 16)
pico_input_gp13: Digital INput GP13 (Pico pin 17)


## Outputs

Let's try some outputs. This code should set all four outputs high (3.3V).

In [6]:
pico.output_gp19(True)
pico.output_gp20(True)
pico.output_gp21(True)
pico.output_gp22(True)

This code resets the outputs to low (0V).

In [7]:
pico.output_gp19(False)
pico.output_gp20(False)
pico.output_gp21(False)
pico.output_gp22(False)

## Inputs

Inputs start in the disabled state.
That's not to say they don't work,
but depending on electrical conditions,
you may not get what you expect.

In [5]:
print(pico.inputs_enabled.__doc__+"\n")
print(f"Enabled: {pico.inputs_enabled.get()}")

Enables or disables the use of inputs

Parameter class:

* `name` inputs_enabled
* `label` inputs_enabled
* `unit` 
* `vals` <Boolean>

Enabled: False


If you're going to use the inputs,
its best to explicitly enable them.

In [6]:
pico.inputs_enabled(True)
print(pico.inputs_enabled.get())

True


Try pressing setting some combination of inputs and reading their state.

In [4]:
print(pico.input_gp10.get())
print(pico.input_gp11.get())
print(pico.input_gp12.get())
print(pico.input_gp13.get())

True
True
False
True


If you're using the breadboard design, this code should illustrate the switches' state.

In [6]:
def render(parameter):
    return "CLOSED" if parameter.get() else "OPEN  "

print(f"""{render(pico.input_gp10)}  {render(pico.input_gp11)}

{render(pico.input_gp12)}  {render(pico.input_gp13)}""")

OPEN    CLOSED

OPEN    OPEN  


## Automation

The following code shows an example of setting and clearing the outputs in a timed sequence,
until the upper left button is held down.
If you don't have a breadboard,
you can raise GP10 (Pico pin 14).

If you are using the breadboard design, watch the animation closely to reveal the hidden message!

In [1]:
pico.output_gp19(False)
pico.output_gp20(False)
pico.output_gp21(False)
pico.output_gp22(False)

while not pico.input_gp10.get():
    pico.output_gp19(False)
    sleep(0.05)
    pico.output_gp20(False)
    pico.output_gp21(False)
    sleep(0.05)
    pico.output_gp22(False)

    sleep(0.05)

    pico.output_gp19(True)
    sleep(0.05)
    pico.output_gp20(True)
    pico.output_gp21(True)
    sleep(0.05)
    pico.output_gp22(True)
    sleep(0.05)

    sleep(0.5)

pico.output_gp19(False)
pico.output_gp20(False)
pico.output_gp21(False)
pico.output_gp22(False)

NameError: name 'pico' is not defined

If you don't have a breadboard, this might help:

![Video of the Microsoft logo shimmering, and a finger pressing a button to stop it](shiny.gif)

Did you see the message?

That's right! It says "hire me!"